In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.options.display.max_columns = None

import sys
import time
import numpy as np
np.set_printoptions(threshold = sys.maxsize)

import xgboost as xgb
print(xgb.__version__)

from joblib import dump

# from sklearn.model_selection import cross_val_score, KFold, ShuffleSplit
from sklearn.model_selection import GridSearchCV, train_test_split

2.0.3


In [2]:
mon_laun_train_path = "datasets/merged_dataset_train_done.csv"
# mon_laun_train_df = pd.read_csv(mon_laun_train_path, index_col = False, nrows = 90000)
mon_laun_train_df = pd.read_csv(mon_laun_train_path, index_col = False)
# display(mon_laun_train_df)

In [3]:
mon_laun_train_df_target = mon_laun_train_df[["Is Laundering"]].astype(int)
mon_laun_train_df_data = mon_laun_train_df.drop(columns = ["Is Laundering"])

display(mon_laun_train_df_target.shape)
display(mon_laun_train_df_data.shape)

drop_columns = ['Timestamp', \
                'sourceAccountID', 'sourceAccount', 'From Bank', 'Account', \
                'targetAccountID', 'targetAccount', 'To Bank', 'Account.1', \
                'Receiving Currency', 'Payment Currency', 'Payment Format'    
               ]

# remaing columns ['transactionID', 'Timestamp_float', \
#                 'Amount Received', 'Receiving CurrencyID', \
#                 'Amount Paid', 'Payment CurrencyID', \
#                 'Payment FormatID'
#                 ]
# NOTE: graph and vertex feature are included

mon_laun_train_df_data = mon_laun_train_df_data.drop(columns = drop_columns)

(4062676, 1)

(4062676, 265)

In [4]:
display(mon_laun_train_df_target.dtypes)
display(mon_laun_train_df_data.dtypes)
# display(mon_laun_train_df_data)

X_train = mon_laun_train_df_data.to_numpy()
y_train = mon_laun_train_df_target.to_numpy()

Is Laundering    int64
dtype: object

transactionID             float64
Timestamp_float           float64
Amount Received           float64
Receiving CurrencyID      float64
Amount Paid               float64
                           ...   
dest_max_col11_in         float64
dest_median_col11_in      float64
dest_var_col11_in         float64
dest_skew_col11_in        float64
dest_kurtosis_col11_in    float64
Length: 253, dtype: object

In [5]:
X_train_, X_val_, y_train_, y_val_ = train_test_split(X_train, y_train, 
                                          test_size = 0.25, 
                                          random_state = 52, 
                                          shuffle = False, 
                                          stratify = None
                                         )

val_indices_ = np.arange(len(X_train_), len(X_train_) + len(X_val_))

In [6]:
print(sum(y_train_))
print(sum(y_val_))

[2298]
[1082]


In [7]:
params = {
    # NOT DOCUMENTED in XGBoost
    # 'n_estimators': [10, 100, 500, 1000],
        # 'early_stopping_rounds': [1500], # ERROR: Must have at least 1 validation dataset for early stopping

    # DOCUMENTED in XGBoost
        # 'num_round': [i for i in range(10, 1000, 10)], # in console only --> TODO: NOT USED
        # 'base_score': # global bias
    
    # 'seed': [i for i in range(1, 10)],
        # 'seed_per_iteration':
        
    # 'learning_rate': [i for i in np.arange(10**(-2.5), 10**(-1), 0.01)], # eta --> [0.03, 0.1]
    # 'reg_lambda': [i for i in np.arange(10**(-2), 10**(2), 10)], # lambda --> [0.01, 100]
        # 'reg_alpha': [i for i in np.arange()], # alpha
        # 'min_split_loss': [] # gamma
    
    # 'max_depth': [i for i in range(1, 15)], 
        # 'min_child_weight': [1, 2, 3, 4], 
        # 'max_delta_step': [0, 1, 2, 4, 8], 
    # 'subsample': [i for i in np.arange(0.5, 1.0, 0.1)], # to grow a tree
        # 'tree_method': [] # tree construction algorithm

    # 'scale_pos_weight': [i for i in range(1, 10)], # balance of positive and negative weights, useful for unbalanced classes
        # 'max_cat_to_onehot', 'max_cat_threshold'
        # 'updater', 'refresh_leaf', 'process_type', 'max_leaves', 'max_bin', 
        # 'num_parallel_tree', 'monotone_constraints', 'interaction_constraints', 'multi_strategy', 'max_cached_hist_node'
    
    # 'colsample_bytree': [i for i in np.arange(0.5, 1.0, 0.1)], 
        # 'colsample_bylevel': [i for i in np.arange()],
        # 'colsample_bynode': [i for i in np.arange()]
}

In [8]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

clf = GridSearchCV(estimator = xgb.XGBClassifier(booster = 'gbtree', # gbtree, gblinear, dart
                                                 objective = 'binary:logistic', # 'binary:logistic', 'multi:softprob'
                                                 eval_metric = 'error@0.5', # error@0.5, error, auc, aucpr --> NOTE: during training
                                                 validate_parameters = True,
                                                 random_state = 42,
                                                 device = 'cpu',
                                                 nthread = 6, # default is maximum threads 
                                                 # verbose = 4,
                                                ), 
                    param_grid = params,
                    # https://scikit-learn.org/stable/modules/model_evaluation.html          
                    scoring = 'f1', # accuracy, etc. --> [multi-metric scoring] --> NOTE: after model fit
                    refit = False,
                    cv = [(val_indices_, val_indices_)], # NOTE: check for correctness
                    n_jobs = 6, # 12
                    return_train_score = False
                  )

start = time.time()
clf.fit(X_train, y_train)
end = time.time()
print("CLF fit wait time: " + str(end - start) + " sec")


# /usr/local/anaconda3/envs/money_fraud_project/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak. warnings.warn(

CLF fit wait time: 76.68399834632874 sec


In [9]:
display(clf.best_params_)
# display(clf.best_score_)
# display(clf.n_splits_)  # default 5

# display(clf.best_estimator_) # if refit = True
# display(clf.cv_results_) # if return_train_score = True

{}

In [10]:
xgb_model = xgb.XGBClassifier(**clf.best_params_)
    # params - passes a single dictionary as the first positional arg
    # **params - keys in the dictionary are each passed as keyword args

xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [11]:
dump(xgb_model, "xgb_best_model_params.joblib") # NOTE: saving the best params for later useage

['xgb_best_model_params.joblib']